### Voice Translation of Audio files 

Ever wanted to translate a podcast into your native language? In this cookbook we will walk through steps to dub an audio podcast from English to Hindi.   

The overall steps in the process are: 
1. Transcribe the audio file into text with Whisper 
2. Translate the text in language of your choice with GPT 
3. Convert the translated text to audio with tts (text-to-speech) 
4. Translation benchmarking (BLEU or ROUGE) 


Step 1: Transcribe the audio file into text with Whisper 



In [11]:
audio_file = "../gpt4o/data/keynote_recap.mp3"

from openai import OpenAI
client = OpenAI()

audio_file= open(audio_file, "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1",  
  file=audio_file
)
print(transcription.text)



Welcome to our first-ever OpenAI Dev Day. Today, we are launching a new model, GPT-4 Turbo. GPT-4 Turbo supports up to 128,000 tokens of context. We have a new feature called JSON mode, which ensures that the model will respond with valid JSON. You can now call many functions at once, and it'll do better at following instructions in general. You want these models to be able to access better knowledge about the world. So do we. So we're launching retrieval in the platform. You can bring knowledge from outside documents or databases into whatever you're building. GPT-4 Turbo has knowledge about the world up to April of 2023, and we will continue to improve that over time. Dolly 3, GPT-4 Turbo with Vision, and the new Text-to-Speech model are all going into the API today. Today, we're launching a new program called Custom Models. With Custom Models, our researchers will work closely with the company to help them make a great custom model, especially for them and their use case using our t

In [8]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You are an assistant that translates text from English to Hindi. User will provide content for you to translate. You may keep certain words in English for which a direct translation doesn't exist. Some words to keep in English include - Turbo, OpenAI, token, GPT, Dall-e, Python"},
    {"role": "user", "content": transcription.text},
  ]
)

message = response.choices[0].message.content

print(message)

हमारे पहले OpenAI Dev Day में आपका स्वागत है। आज, हम एक नया मॉडल, GPT-4 Turbo लॉन्च कर रहे हैं। GPT-4 Turbo 128,000 tokens तक के context का समर्थन करता है। हमारे पास JSON mode नाम का एक नया फीचर है, जो सुनिश्चित करता है कि मॉडल वैध JSON के साथ प्रतिक्रिया देगा। अब आप कई functions को एक साथ कॉल कर सकते हैं, और यह सामान्य रूप से निर्देशों का पालन करने में बेहतर करेगा। आप चाहते हैं कि ये मॉडल दुनिया के बारे में बेहतर ज्ञान प्राप्त कर सकें। हम भी ऐसा ही चाहते हैं। इसलिए हम प्लेटफॉर्म में retrieval लॉन्च कर रहे हैं। आप बाहरी दस्तावेजों या डेटाबेस से ज्ञान को अपने निर्माण में ला सकते हैं। GPT-4 Turbo के पास अप्रैल 2023 तक की दुनिया के बारे में ज्ञान है, और हम इसे समय के साथ सुधारते रहेंगे। Dolly 3, GPT-4 Turbo with Vision, और नया Text-to-Speech मॉडल सभी आज API में जा रहे हैं। आज, हम Custom Models नामक एक नया प्रोग्राम लॉन्च कर रहे हैं। Custom Models के साथ, हमारे शोधकर्ता कंपनी के साथ मिलकर काम करेंगे ताकि वे उनके विशेष उपयोग के मामले के लिए एक महान कस्टम मॉडल बना सकें, जो हमारे टूल्स और उच्

In [23]:
from openai import OpenAI

client = OpenAI()

response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input=message,
)

response.stream_to_file("output.mp3")

/var/folders/jk/xl1ppdk94637355tx59m0kph0000gp/T/ipykernel_9722/2703525150.py:11: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file("output.mp3")


In [24]:
from playsound import playsound

playsound("output.mp3")

In [ ]:
# Checking the quality of translation 
